## Install Dependencies

In [ ]:
!pip install pandas
!pip install numpy
!pip install tensorflow
!pip install scikit-learn

  Using cached pandas-2.3.3-cp313-cp313-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (91 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.3.3-cp313-cp313-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl (12.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 186.6 kB/s  0:01:32m0:00:0100:04
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached pillow-12.0.0-cp3

## Imports

In [ ]:
import pandas as pd
import numpy as np
import re
import pickle
from tqdm.auto import tqdm
tqdm.pandas()

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
)

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

## Loading Dataset

In [ ]:
print("🔄 Loading dataset...")

fake_df = pd.read_csv("dataset/fake-and-real-news-dataset/Fake.csv")
true_df = pd.read_csv("dataset/fake-and-real-news-dataset/True.csv")

print("✔ Loaded successfully!")
print("Fake shape:", fake_df.shape)
print("True shape:", true_df.shape)

: 

## Assigning Labels

In [ ]:
print("\n🔄 Assigning labels (0 = Fake, 1 = True)...")

fake_df["label"] = 0
true_df["label"] = 1

df = pd.concat([fake_df, true_df], axis=0).reset_index(drop=True)

print("✔ Dataset combined. Total records:", df.shape[0])

In [ ]:
# 3. CLEANING TEXT
# ============================================================

print("\n🔄 Cleaning text... (this may take time)")

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text)            # Punctuation/numbers
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df["clean_text"] = df["text"].progress_apply(clean_text)

print("✔ Text cleaning completed.")

In [ ]:
# 4. SPLIT DATA
# ============================================================

print("\n🔄 Splitting data into train/test sets...")

X = df["clean_text"].values
y = df["label"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("✔ Data split complete.")
print("Training samples:", len(X_train))
print("Testing samples:", len(X_test))

In [ ]:
# 5. TOKENIZATION + SEQUENCING
# ============================================================

print("\n🔄 Tokenizing and converting to sequences...")

vocab_size = 20000
max_length = 300

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(tqdm(X_train))

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_length)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length)

print("✔ Tokenization complete. Vocabulary size:", len(tokenizer.word_index))

In [ ]:
# 6. SAVE TOKENIZER
# ============================================================

print("\n💾 Saving tokenizer...")

with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

print("✔ Tokenizer saved as tokenizer.pkl")

In [ ]:
# 7. BUILD CNN MODEL
# ============================================================

print("\n🔄 Building CNN model...")

model = Sequential([
    Embedding(vocab_size, 128, input_length=max_length),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),

    Dense(64, activation='relu'),
    Dropout(0.3),

    Dense(1, activation='sigmoid')
])

model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

model.summary()
print("✔ CNN model built.")

In [ ]:
# 8. TRAIN MODEL
# ============================================================

print("\n🚀 Training CNN model... (this WILL take several minutes)")

history = model.fit(
    X_train_pad, y_train,
    epochs=5,
    batch_size=128,
    validation_split=0.1,
    verbose=1
)

print("✔ Model training complete.")

In [ ]:
# 9. EVALUATION
# ============================================================

print("\n📊 Evaluating model...")

y_pred = (model.predict(X_test_pad) > 0.5).astype("int32")

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
# 10. SAVE MODEL

print("\n💾 Saving model...")

model.save("fake_news_cnn.h5")

print("✔ Model saved as fake_news_cnn.h5")
print("\n🎉 ALL DONE — CNN model is ready for deployment!")